In [1]:
import torch as t
import numpy as np
from pathlib import Path

In [2]:
device = t.device("mps" if t.backends.mps.is_available() else "cuda" if t.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
root_dir = Path().cwd().parent.parent.parent
data_dir = root_dir / "data"
keys = ["boards", "histories", "legal_moves"]
dataset = {k: np.load(data_dir / f"othello6-{k}.npy") for k in keys}

In [4]:
model_cfg = dict(
    n_layers=8,
    d_model=512,
    d_head=64,
    n_heads=8,
    d_mlp=2048,
    d_vocab=61,
    n_ctx=59,
    act_fn="gelu",
    normalization_type="LNPre",
    device=device,
)

{'boards': array([[[[ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  1,  1,  1,  0],
          [ 0,  0, -1,  1,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0]],
 
         [[ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0, -1,  0],
          [ 0,  0,  1, -1,  1,  0],
          [ 0,  0, -1,  1,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0]],
 
         [[ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0, -1,  0],
          [ 0,  0,  1, -1,  1,  0],
          [ 0,  0,  1,  1,  0,  0],
          [ 0,  0,  1,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0]],
 
         ...,
 
         [[ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0]],
 
         [[ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,  0,  0],
          [ 0,  0,  0,  0,